In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2

In [2]:
df = pd.read_csv('spam_assassin.csv')

In [3]:
df.head()

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


In [4]:
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def email_body_extractor(text):
    # Use BeautifulSoup to parse HTML content
    soup = BeautifulSoup(text, 'lxml')
    # Extract the text content of the email
    email_body = soup.get_text()
    # Remove email addresses and URLs
    stripped = re.sub(combined_pat, '', email_body)
    try:
        # Handle utf-8 encoding issues
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    # Remove non-alphabetic characters
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    # Convert to lowercase
    lower_case = letters_only.lower()
    # Tokenize and remove stopwords
    words = tok.tokenize(lower_case)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Join the words back into a string
    return (" ".join(words)).strip()

# Example usage with a dataset named 'df'
# Replace 'df.text' with the actual column containing the email text in your dataset
testing = df.text[:10]
test_result = [email_body_extractor(t) for t in testing]
print(test_result)

['ilug admin ie mon jul return path delivered yyyy netnoteinc com received localhost localhost phobos labs netnoteinc com postfix esmtp id f mon jul edt received phobos localhost imap fetchmail jm single drop mon jul ist received lugh tuatha org root tuatha org dogma slashnull org esmtp id g rhn sat jul received lugh root lugh tuatha org esmtp id saa sat jul x authentication warning lugh tuatha org host root claimed lugh received mail mail iol ie mail mail iol ie lugh tuatha org esmtp id saa sat jul received dialup ts cwt esat net helo hobbiton cod ie mail mail iol ie esmtp exim id yvvf w ilug ie sat jul received cdaly hobbiton cod ie id g rdroo ilug ie sat jul date sat jul conor daly ilug main list subject ilug architecture crossover trouble w rh solved message id b cod ie mail followup ilug main list references c dce cd b c ba e fa maxtor com mime version content type text plain charset us ascii content disposition inline user agent mutt reply c dce cd b c ba e fa maxtor com conor wy

In [6]:
nums = [0,5795]
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%100 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_tweet_texts.append(email_body_extractor(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 100 of 5795 has been processed
Tweets 200 of 5795 has been processed
Tweets 300 of 5795 has been processed
Tweets 400 of 5795 has been processed
Tweets 500 of 5795 has been processed
Tweets 600 of 5795 has been processed
Tweets 700 of 5795 has been processed
Tweets 800 of 5795 has been processed
Tweets 900 of 5795 has been processed
Tweets 1000 of 5795 has been processed
Tweets 1100 of 5795 has been processed
Tweets 1200 of 5795 has been processed
Tweets 1300 of 5795 has been processed
Tweets 1400 of 5795 has been processed
Tweets 1500 of 5795 has been processed
Tweets 1600 of 5795 has been processed
Tweets 1700 of 5795 has been processed
Tweets 1800 of 5795 has been processed
Tweets 1900 of 5795 has been processed
Tweets 2000 of 5795 has been processed
Tweets 2100 of 5795 has been processed
Tweets 2200 of 5795 has been processed
Tweets 2300 of 5795 has been processed
Tweets 2400 of 5795 has been processed
Tweets 2500 of 5795 has been processe

In [7]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['label'] = df.target
clean_df.head()

,text,label
0,ilug admin ie mon jul return path delivered yy...,0
1,gort com mon jun return path gort com delivery...,1
2,fork admin com mon jul return path delivered y...,1
3,dcm net cn mon jun return path dcm net cn deli...,1
4,ilug admin ie mon aug return path delivered yy...,0


In [8]:
clean_df.to_csv('email_classification.csv',encoding='utf-8')
#df = pd.read_csv('dataset.csv', encoding='latin-1', header=None, names=cols)
df = pd.read_csv('email_classification.csv')

In [9]:
df.head()

,Unnamed: 0,text,label
0,0,ilug admin ie mon jul return path delivered yy...,0
1,1,gort com mon jun return path gort com delivery...,1
2,2,fork admin com mon jul return path delivered y...,1
3,3,dcm net cn mon jun return path dcm net cn deli...,1
4,4,ilug admin ie mon aug return path delivered yy...,0


In [10]:
import pandas as pd

# Assuming 'your_file.csv' is your CSV file with unnamed columns
df = pd.read_csv('email_classification.csv')

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Find and drop unnamed columns
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns, errors='ignore')

# Display the DataFrame after dropping unnamed columns
print("\nDataFrame after dropping unnamed columns:")
print(df)

# Now you can save the modified DataFrame back to a CSV file if needed
# df.to_csv('modified_file.csv', index=False)


Original DataFrame:
      Unnamed: 0                                               text  label
0              0  ilug admin ie mon jul return path delivered yy...      0
1              1  gort com mon jun return path gort com delivery...      1
2              2  fork admin com mon jul return path delivered y...      1
3              3  dcm net cn mon jun return path dcm net cn deli...      1
4              4  ilug admin ie mon aug return path delivered yy...      0
...          ...                                                ...    ...
5790        5790  crackmice admin com thu sep return path delive...      1
5791        5791  ilug admin ie mon jul return path delivered yy...      0
5792        5792  fork admin com mon oct return path delivered y...      0
5793        5793  received hq pro ns net localhost hq pro ns net...      1
5794        5794  razor users admin sourceforge net thu sep retu...      0

[5795 rows x 3 columns]

DataFrame after dropping unnamed columns:
            

In [11]:
import pandas as pd

# Assuming 'your_file.csv' is your CSV file with unnamed columns
input_file = 'email_classification.csv'
output_file = 'email_classification1.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Find and drop unnamed columns
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns, errors='ignore')

# Display the DataFrame after dropping unnamed columns
print("\nDataFrame after dropping unnamed columns:")
print(df)

# Save the modified DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"\nModified DataFrame saved to {output_file}")


Original DataFrame:
      Unnamed: 0                                               text  label
0              0  ilug admin ie mon jul return path delivered yy...      0
1              1  gort com mon jun return path gort com delivery...      1
2              2  fork admin com mon jul return path delivered y...      1
3              3  dcm net cn mon jun return path dcm net cn deli...      1
4              4  ilug admin ie mon aug return path delivered yy...      0
...          ...                                                ...    ...
5790        5790  crackmice admin com thu sep return path delive...      1
5791        5791  ilug admin ie mon jul return path delivered yy...      0
5792        5792  fork admin com mon oct return path delivered y...      0
5793        5793  received hq pro ns net localhost hq pro ns net...      1
5794        5794  razor users admin sourceforge net thu sep retu...      0

[5795 rows x 3 columns]

DataFrame after dropping unnamed columns:
            

In [12]:
df = pd.read_csv('email_classification1.csv')

In [13]:
df.head()

,text,label
0,ilug admin ie mon jul return path delivered yy...,0
1,gort com mon jun return path gort com delivery...,1
2,fork admin com mon jul return path delivered y...,1
3,dcm net cn mon jun return path dcm net cn deli...,1
4,ilug admin ie mon aug return path delivered yy...,0


In [14]:
df.label.value_counts()

label
0    3899
1    1896
Name: count, dtype: int64

In [15]:
import pandas as pd

# Load your CSV file into a DataFrame
df = pd.read_csv('email_classification1.csv')

# Assuming your DataFrame has a 'class' column indicating the class for each data point
# You can adjust the column name based on your actual dataset structure
classes = df['label'].unique()

# Set the desired number of samples per class
samples_per_class = 1896  # Adjust this value based on your requirements

# Create an empty DataFrame to store the selected data
selected_data = pd.DataFrame()

# Iterate through each class and select the desired number of samples
for class_label in classes:
    # Filter the DataFrame to include only rows with the current class
    class_data = df[df['label'] == class_label].sample(samples_per_class, random_state=42)
    
    # Append the selected data to the new DataFrame
    selected_data = selected_data._append(class_data)

# Save the selected data to a new CSV file
selected_data.to_csv('email_classification2.csv', index=False)

In [16]:
df = pd.read_csv('email_classification2.csv')
df.label.value_counts()

label
0    1896
1    1896
Name: count, dtype: int64